In [1]:
import numpy as np
import os,sys
from PIL import Image
import re
import matplotlib.image as mpimg

from time import time
from  sklearn.ensemble import GradientBoostingRegressor
import mask_to_submission


# Please indicate the path to the data

In [2]:
path_to_data = ""

In [3]:
# Loaded a set of images
root_dir = path_to_data + "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)[1:]
n = len(files)
print("Loading " + str(n) + " images")
imgs = np.array([mpimg.imread(image_dir + files[i]) for i in range(n)])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = np.array([mpimg.imread(gt_dir + files[i]) for i in range(n)])

Loading 100 images
Loading 100 images


# Gradient boosting regression

In [4]:
x_train = np.array(imgs)
y_train = np.array(gt_imgs)


st = time()

model = GradientBoostingRegressor(n_estimators=500)
model.fit(x_train.reshape(-1,3),y_train.reshape(-1))

print time() - st

17.124311924


# Testset's importation

In [5]:
root_dir = path_to_data + "test_set_images/"

n = 51
print("Loading " + str(n) + " images")
imgs = np.array([mpimg.imread(root_dir + "test_"+str(i)+"/" + "test_"+str(i)+".png") for i in range(1,n)])

Loading 51 images


# classification prediction

In [6]:
pred_test = model.predict(imgs.reshape(-1, 3))
pred_test = pred_test.reshape(imgs.shape[:-1])

# Segmentation with radom_walker

In [7]:
from skimage.segmentation import random_walker
from sklearn.metrics import f1_score
def pred_random_walker(reg, thr_non_road, thr_road, beta):
    '''
    Process random walker algorithm for every images.
    
    reg: np.ndarray(number of image, number of row,  number of column)
    thr_road: minimum value for being consider as a road
    thr_non_road: minimum value for being consider as a non-road
    beta: random walker's penalization
    '''
    segmentation = np.zeros_like(reg)
    for i in range(len(reg)):
        data = reg[i]
        
        markers=np.zeros_like(data)
        markers[data<thr_non_road] = 1
        markers[data>thr_road] = 2
        segmentation[i] = random_walker(data, markers, beta=beta, mode='bf')
    
    segmentation = segmentation -1
    return segmentation

In [14]:
image_shape = imgs.shape[:-1]
segmentations = pred_random_walker(pred_test.reshape(image_shape), 0.05, 0.5, beta=500)

# Write the segmentations from the random walker algorithm in files.

In [15]:
# creation of the test masks folder if not exist
try:
    os.stat(path_to_data + 'test_set_masks')
except:
    os.mkdir(path_to_data + 'test_set_masks')       


In [16]:
import scipy.misc
def write_segmentation( segmentations):
    for i in range(1, len(segmentations)+1):
        filename =  path_to_data + 'test_set_masks/satImage_' + '%.3d' % i + '.png'
        scipy.misc.imsave(filename, segmentations[i-1])

In [17]:
write_segmentation( segmentations)

# make the submission file

In [18]:
submission_filename = 'baseline_submission.csv'
image_filenames = []

for i in range(1, 51):
    image_filename = path_to_data + 'test_set_masks/satImage_' + '%.3d' % i + '.png'
    image_filenames.append(image_filename)
mask_to_submission.masks_to_submission(submission_filename, *image_filenames)